In [2]:
def divided_differences(x_values, y_values, k):
    result = 0
    for j in range(k + 1):
        mul = 1
        for i in range(k + 1):
            if i != j:
                mul *= x_values[j] - x_values[i]
        result += y_values[j]/mul
    return result


def create_Newton_polynomial(x_values, y_values):
    div_diff = []
    for i in range(1, len(x_values)):
        div_diff.append(divided_differences(x_values, y_values, i))
    def newton_polynomial(x):
        result = y_values[0]
        for k in range(1, len(y_values)):
            mul = 1
            for j in range(k):
                mul *= (x-x_values[j])
            result += div_diff[k-1]*mul
        return result
    return newton_polynomial

def function(x):
    return pow(7,5*x)

def main():
    start = float(input("a = "))
    end = float(input("b = "))
    amount_of_dots = 20
    x_list = [ start + i * ((end - start)/(amount_of_dots-1)) for i in range(amount_of_dots)] 
    y_list = [function(x_list[i]) for i in range(amount_of_dots)] 
    new_pol = create_Newton_polynomial(x_list, y_list)

    for x in x_list:
        print("x = %f y1 = %s" % (x,("yes" if abs(new_pol(x)-function(x)) < 0.000001 else "no")))
        if(abs(new_pol(x)-function(x)) > 0.000001):
            pass

main()

x = 1.000000 y1 = yes
x = 1.263158 y1 = yes
x = 1.526316 y1 = yes
x = 1.789474 y1 = yes
x = 2.052632 y1 = yes
x = 2.315789 y1 = yes
x = 2.578947 y1 = no
x = 2.842105 y1 = no
x = 3.105263 y1 = no
x = 3.368421 y1 = yes
x = 3.631579 y1 = no
x = 3.894737 y1 = no
x = 4.157895 y1 = no
x = 4.421053 y1 = no
x = 4.684211 y1 = yes
x = 4.947368 y1 = no
x = 5.210526 y1 = no
x = 5.473684 y1 = no
x = 5.736842 y1 = yes
x = 6.000000 y1 = no


In [1]:
#polynomial_stuff.py

import numpy as np

def get_y_values(x):
    y = []
    for i in range(len(x)):
        y.append(np.sin(x[i]/2) + np.cbrt(x[i]))
    return y

def newtons_polinomial(x, y):
    div_diffs = [y]
    step = 0
    for j in range(1, len(x)):
        div_diffs.append([])
        for i in range(0, len(x) - 1 - step):
            div_diffs[j].append((div_diffs[j-1][i] - div_diffs[j-1][i+1])
                                 /(x[i] - x[i+j]))
        step+=1
    return div_diffs

def print_polynomial(div_diffs, x):
    print("Newton's polynomial:")
    for i in range(len(x)):
        print(f'{div_diffs[i][0]:.6f}', end = '')
        for j in range(i):
            if x[j] >= 0:
                print(f'*(x-{x[j]})', end = '')
            else:
                print(f'*(x+{abs(x[j])})', end = '')
        print(' ', end = '')
        if i != len(x)-1 and div_diffs[i+1][0] > 0:
            print('+', end = '')
    print()

def cub_spline(x, y):
    a = []
    b = []
    d = []
    h = []
    for i in range(1, len(x)):
        a.append(y[i-1])
        h.append(x[i] - x[i-1])
    
    vector_b = []
    vector_d = []
    for i in range(1, len(x)-1):### ## -2
        vector_b.append(2*(h[i-1]+h[i]))
        vector_d.append(6*((y[i+1]-y[i])/h[i] - (y[i]-y[i-1])/h[i-1]))
    vector_a = h[:len(x)-3]
    vector_a.insert(0, 0)
    vector_c = h[1:len(x)-2]
    vector_c.insert(len(h), 0)

    c = progon(vector_a, vector_b, vector_c, vector_d)
    c.insert(0, 0)
    c.insert(len(c), 0)
    for i in range(len(x) - 1):
        if i == len(x) - 2:
            d.append(-1*c[len(x)-2]/(3*h[i]))
            b.append(-2*h[i]*c[i]/3 + (y[i+1]-y[i])/h[i])
        else:
            d.append((c[i+1] - c[i])/(3*h[i]))
            b.append((y[i+1]-y[i])/h[i] - c[i]*h[i] - d[i]*h[i]*h[i])
    return a, b, c, d

def progon(a, b, c, d):
    A = []
    B = []
    A.append(-c[1]/b[1])
    B.append(d[1]/b[1])
    for i in range(1, len(d)):
        e = a[i]*A[i-1]+b[i]
        A.append(-c[i]/e)
        B.append((d[i]-a[i]*B[i-1])/e)
    x = []
    x.insert(0, B[len(d)-1])
    for i in range(len(d)-2, -1, -1):
            x.insert(0, A[i]*x[0] + B[i])
    return x

def print_splines(a, b, c, d, x):
    print("Cubic spline polynomials:")
    for i in range (len(a)):
        
        pars = f"(x-{x[i]})" if x[i] >= 0 else f"(x+{abs(x[i])})"
        print(f'{a[i]:-9.6f} ', end = '')

        print(f'+', end = '') if b[i] > 0 else print(f'-', end = '')
        print(f'{abs(b[i]):-9.6f}{pars} ', end = '')

        print(f'+', end = '') if c[i] > 0 else print(f'-', end = '')
        print(f'{abs(c[i]):-9.6f}{pars}^2 ', end = '')

        print(f'+', end = '') if d[i] > 0 else print(f'-', end = '')
        print(f'{abs(d[i]):-9.6f}{pars}^3 for x in [{x[i]:2};{x[i+1]:2}]')


def function(x):
    return 3*x*x+2*x+1

def main():
    start = float(input("a = "))
    end = float(input("b = "))
    amount_of_dots = 10
    x_list = [ start + i * ((end - start)/(amount_of_dots-1)) for i in range(amount_of_dots)] 
    y_list = [function(x_list[i]) for i in range(amount_of_dots)] 
    coeffs = newtons_polinomial(x_list, y_list)
    print_polynomial(coeffs, x_list)
    a, b, c, d = cub_spline(x_list, y_list)
    print_splines(a, b, c, d, x_list)

main()

Newton's polynomial:
6.000000 +11.000000*(x-1.0) +3.000000*(x-1.0)*(x-2.0) -0.000000*(x-1.0)*(x-2.0)*(x-3.0) -0.000000*(x-1.0)*(x-2.0)*(x-3.0)*(x-4.0) -0.000000*(x-1.0)*(x-2.0)*(x-3.0)*(x-4.0)*(x-5.0) -0.000000*(x-1.0)*(x-2.0)*(x-3.0)*(x-4.0)*(x-5.0)*(x-6.0) -0.000000*(x-1.0)*(x-2.0)*(x-3.0)*(x-4.0)*(x-5.0)*(x-6.0)*(x-7.0) -0.000000*(x-1.0)*(x-2.0)*(x-3.0)*(x-4.0)*(x-5.0)*(x-6.0)*(x-7.0)*(x-8.0) -0.000000*(x-1.0)*(x-2.0)*(x-3.0)*(x-4.0)*(x-5.0)*(x-6.0)*(x-7.0)*(x-8.0)*(x-9.0) 
Cubic spline polynomials:
 6.000000 + 8.464151(x-1.0) - 0.000000(x-1.0)^2 + 2.535849(x-1.0)^3 for x in [1.0;2.0]
17.000000 +10.071698(x-2.0) + 7.607547(x-2.0)^2 - 0.679245(x-2.0)^3 for x in [2.0;3.0]
34.000000 +17.249057(x-3.0) + 5.569811(x-3.0)^2 + 0.181132(x-3.0)^3 for x in [3.0;4.0]
57.000000 +22.932075(x-4.0) + 6.113208(x-4.0)^2 - 0.045283(x-4.0)^3 for x in [4.0;5.0]
86.000000 +29.022642(x-5.0) + 5.977358(x-5.0)^2 + 0.000000(x-5.0)^3 for x in [5.0;6.0]
121.000000 +34.977358(x-6.0) + 5.977358(x-6.0)^2 + 0.0452

Invalid input
Invalid input
Invalid input
Maximum is  55
Minimum is  55
